# **08-02 CNN으로 MNIST 분류하기**

- https://wikidocs.net/63565

## 1. 모델 이해하기

- **합성곱 층 표기 방법 1**:  
  - 합성곱(`nn.Conv2d`) + 활성화 함수(`nn.ReLU`) → 합성곱 층  
  - 맥스풀링(`nn.MaxPool2d`) → 별도의 풀링 층으로 구분  
- **합성곱 층 표기 방법 2**:  
  - 합성곱(`nn.Conv2d`) + 활성화 함수(`nn.ReLU`) + 맥스풀링(`nn.MaxPool2d`) → 하나의 합성곱 층으로 묶음 

- 모델의 아키텍처

```py
# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 7 × 7 × 64 → batch_size × 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax
```

## 2. 모델 구현하기

- 합성곱(`nn.Conv2d`) + 활성화 함수(`nn.ReLU`) + 맥스풀링(`nn.MaxPool2d`) → 하나의 합성곱 층으로 묶음 

1. 필요한 도구 임포트와 입력의 정의

In [4]:
# 필요한 도구 임포트 
import torch
import torch.nn as nn

In [6]:
# 배치 크기 x 채널 x 높이(height) x 너비(width)의 크기의 임의의 텐서 선언 
inputs = torch.Tensor(1, 1, 28, 28)
print("텐서의 크기 : {}", format(inputs.shape))

텐서의 크기 : {} torch.Size([1, 1, 28, 28])


2. 합성곱층과 풀링 선언하기

- 첫번째 합성곱 층
    - 1채널 짜리를 입력받아서 32채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1

In [8]:
conv1 = nn.Conv2d(1, 32, 3, padding=1)
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

- 두번째 합성곱 층
    - 32채널 짜리를 입력받아서 64채널을 뽑아내는데 커널 사이즈는 3이고 패딩은 1

In [65]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

- 맥스풀링
    - 정수 하나를 인자로 넣으면 커널 사이즈와 스트라이드가 둘 다 해당값으로 지정

In [12]:
pool = nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

3. 구현체를 연결하여 모델 만들기

- 첫번째 합성곱층 통과
    - 32가 나온 이유는 conv1의 out_channel로 32를 지정해주었기 때문
    - 28너비 28높이가 된 이유는 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존되기 때문

In [76]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [16]:
# 맥스풀링 통과
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

- 두번째 합성곱층에 통과
    - 64가 나온 이유는 conv2의 out_channel로 64를 지정해주었기 때문
    - 또한, 14너비 14높이가 된 이유는 패딩을 1폭으로 하고 3 × 3 커널을 사용하면 크기가 보존되기 때문

In [18]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [20]:
# 맥스풀링 통과
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

- 텐서를 펼치는 작업
- 현재 out의 크기는 1 × 64 × 7 × 7

In [22]:
out.size(0)

1

In [24]:
out.size(1)

64

In [26]:
out.size(2)

7

In [28]:
out.size(3)

7

- .view()를 사용하여 텐서를 펼치는 작업

In [30]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
out.shape

torch.Size([1, 3136])

- 전결합층(Fully-Connteced layer)를 통과
    - 출력층으로 10개의 뉴런을 배치하여 10개 차원의 텐서로 변환

In [32]:
fc = nn.Linear(3136, 10) # input_dim = 3,136, output_dim = 10
out = fc(out)
out.shape

torch.Size([1, 10])

## 3. CNN으로 MNIST 분류하기

- 필요한 도구 임포트

In [35]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

- 만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 된다.

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# 랜덤 시드 고정
torch.manual_seed(123)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == "cuda":
    torch.cuda.manual_seed_all(123)

- 학습에 사용할 파라미터 설정

In [39]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

- 데이터로더 사용하여 데이터 다루기 위한 데이터셋을 정의

In [41]:
mnist_train = dsets.MNIST(root="MNIST_data/",  # 다운로드 경로 지정
                          train=True,  # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(),  # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root="MNIST_data/",  # 다운로드 경로 지정
                         train=False,  # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(),  # 텐서로 변환
                         download=True)

100%|██████████████████████████████████████████████████████████████████████████████| 9.91M/9.91M [00:17<00:00, 573kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 151kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 1.65M/1.65M [00:02<00:00, 662kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 4.52MB/s]


- 데이터로더 사용하여 배치 크기 지정

In [45]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

- 클래스 모델 설계

In [47]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

- 모델 정의

In [49]:
# CNN 모델 정의
model = CNN().to(device)

- 비용 함수와 옵티마이저 정의

In [51]:
criterion = torch.nn.CrossEntropyLoss().to(device)  # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

- 총 배치 수 출력

In [53]:
total_batch = len(data_loader)
print("총 배치의 수 : {}".format(total_batch))

총 배치의 수 : 600


- 모델 훈련

In [55]:
for epoch in range(training_epochs):
    avg_cost = 0  # 에포크당 평균 비용을 저장하기 위한 변수 초기화

    for X, Y in data_loader:  # 미니 배치 단위로 데이터를 꺼내옴. X는 입력 데이터, Y는 레이블
        # 이미지 데이터는 이미 (28x28) 크기를 가지므로, 별도의 reshape 필요 없음
        # 레이블 Y는 원-핫 인코딩이 아닌 정수형 클래스 레이블임
        X = X.to(device)  # 입력 데이터를 연산이 수행될 장치로 이동 (예: GPU)
        Y = Y.to(device)  # 레이블을 연산이 수행될 장치로 이동 (예: GPU)

        optimizer.zero_grad()  # 옵티마이저의 기울기 초기화
        hypothesis = model(X)  # 모델을 통해 예측값(hypothesis)을 계산 (순전파 연산)
        cost = criterion(hypothesis, Y)  # 예측값과 실제값 Y 간의 손실(cost) 계산
        cost.backward()  # 역전파 연산을 통해 기울기 계산
        optimizer.step()  # 옵티마이저를 통해 파라미터 업데이트

        avg_cost += cost / total_batch  # 현재 배치의 비용을 전체 배치 수로 나누어 누적

    # 에포크가 끝날 때마다 평균 비용 출력
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.219468355
[Epoch:    2] cost = 0.0644898415
[Epoch:    3] cost = 0.0470146947
[Epoch:    4] cost = 0.0386773683
[Epoch:    5] cost = 0.0315294191
[Epoch:    6] cost = 0.0262777712
[Epoch:    7] cost = 0.0232687611
[Epoch:    8] cost = 0.0186769627
[Epoch:    9] cost = 0.0158442818
[Epoch:   10] cost = 0.0132772075
[Epoch:   11] cost = 0.011272572
[Epoch:   12] cost = 0.010595778
[Epoch:   13] cost = 0.00780200958
[Epoch:   14] cost = 0.00853386428
[Epoch:   15] cost = 0.00646854052


In [57]:
# 학습을 진행하지 않을 것이므로 torch.no_grad() 사용
with torch.no_grad():
    # 테스트 데이터를 모델에 입력하기 위한 준비
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)  # 테스트 데이터셋의 크기를 맞추고, 연산을 위한 장치로 이동
    Y_test = mnist_test.test_labels.to(device)  # 테스트 데이터셋의 레이블을 연산을 위한 장치로 이동

    # 모델 예측 수행
    prediction = model(X_test)  # 테스트 데이터에 대해 모델이 예측한 결과값

    # 예측 결과와 실제 레이블 비교
    correct_prediction = torch.argmax(prediction, 1) == Y_test  # 예측된 클래스와 실제 레이블이 일치하는지 확인

    # 정확도 계산
    accuracy = correct_prediction.float().mean()  # 정확도를 계산하기 위해 일치하는 예측의 평균을 구함
    print('Accuracy:', accuracy.item())  # 정확도를 출력

C:\Users\user\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\anaconda3\Lib\site-packages\torchvision\datasets\mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9879999756813049


- `torch.no_grad()` → 불필요한 기울기 계산 방지, 메모리 효율 증가.  
- 입력과 레이블은 GPU/CPU로 전송 후 모델에 전달.  
- `torch.argmax` → 예측 클래스 결정.  
- MNIST 데이터셋에서 CNN으로 **98% 정확도 달성**.  